In [ ]:
#!pip install scikit-image
#!pip install opencv-python

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import morphology, io
from skimage.transform import rescale, resize, downscale_local_mean

import os

from skimage.segmentation import slic, mark_boundaries
from skimage import feature #for feature extraction

import cv2

import math
from skimage import transform

In [ ]:
images = os.listdir('../data/ISIC_2017/Images_re')
segments = os.listdir('../data/ISIC_2017/custom_masks')
#Greyscaling
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

im = plt.imread('../data/ISIC_2017/Images_re/'+images[4])
mask = plt.imread('../data/ISIC_2017/custom_masks/'+segments[4])

gray = rgb2gray(im)
plt.imshow(gray,cmap='gray')

# Calculating features

In [ ]:
### Area of lesion using segment
# Total size of the image
total = mask.shape[0] * mask.shape[1] 

# Size of mask only
area = np.sum(mask)

# As percentage
print(area/total*100, "%")

plt.imshow(mask, cmap='gray')

In [ ]:
### Calculation perimeter using a brush

## Structural element, that we will use as a "brush" on our mask. The parameter is "brush size"
brush = morphology.disk(3)

# Use this "brush" to erode the image - eat away at the borders

mask_eroded = morphology.binary_erosion(mask, brush)

# Show side by side
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
axes[0].imshow(mask, cmap='gray')
axes[1].imshow(mask_eroded, cmap='gray')
fig.tight_layout()

# Verify the new mask is smaller
new_area = np.sum(mask_eroded)

print(area)
print(new_area)

## As the new area is smaller, the perimeter is calculated by subtracting the og mask from the mask_eroded
# Subtract the two masks from each other to get the border/perimeter
# What is the length of this perimeter = how many 1s? 

image_perimeter = mask - mask_eroded
perimeter = np.sum(image_perimeter)
print(perimeter)

plt.imshow(image_perimeter, cmap='gray')


In [ ]:
### Creating a version showing the lesion on top of the mask

img1 = im.copy()
img1[mask==0] = 0

plt.imshow(img1)

In [ ]:
## We can make a zoomed in version using the mask later if this code is too complicated (also, plagarism?)

# https://stackoverflow.com/questions/59191179/finding-coordinates-of-corners-of-the-maskrectengular-shape-from-mask-matrixb

array = mask.copy()
H,W = array.shape

left_edges = np.where(array.any(axis=1),array.argmax(axis=1),W+1)
flip_lr = cv2.flip(array,1) #1 horz vert 0
right_edges = W - np.where(flip_lr.any(axis=1),flip_lr.argmax(axis=1),W+1)
top_edges = np.where(array.any(axis=0),array.argmax(axis=0),H+1)
flip_ud = cv2.flip(array,0) #1 horz vert 0
bottom_edges = H - np.where(flip_ud.any(axis=0),flip_ud.argmax(axis=0),H+1)

leftmost = left_edges.min()
rightmost = right_edges.max()
topmost = top_edges.min()
bottommost = bottom_edges.max()

In [ ]:
im2 = img1[topmost:bottommost,leftmost:rightmost,:]
mask2 = mask[topmost:bottommost,leftmost:rightmost]
plt.imshow(im2)

In [ ]:
### Asymmetry
## if rotated lesion and lesion are overlapped and there exists a high value of gray, then the lesion is assymetric

#mask2 = morphology.disk(5)

h, w = map(int, mask2.shape)
left = mask2[0:, 0:math.floor(w/2)]
right = mask2[0:, math.ceil(w/2):]
rot_im = transform.rotate(right, 180) ##check if it rotates clockwise or counter clockwise
new_im = rot_im + left
new_im[new_im == 2] = 0

h2, w2 = map(int, new_im.shape)
top = new_im[0: math.floor(h2/2), 0:]
bottom = new_im[math.ceil(h2/2):, 0:]
rot_im2 = transform.rotate(top, 270)
new_im2 = rot_im2 + bottom
new_im2[new_im2 == 2] = 0

plt.imshow(new_im2, cmap='gray')
asymmetry = np.sum(new_im2)/(area)
print(asymmetry)

# number of grey pixels depends on the size of the lesion as well
# so even if the lesion is small, it would still have a lot of gray pixels
# so how would this work as a feature?

In [ ]:
### Measuring shape with area and perimeter
## Compactness = l^2/(4pi*A) --> l = perimeter, A = area

c = (perimeter)**2/(4*math.pi*area)
print(c)

## the minimal the value of c (aka the close it is to 1), the more symmetric (circular) it is
# cannot be 1 perfectly due to pixels

# how is compactness different from symmetry
## why check assymetry? a hexagon is pretty symmetrical

In [ ]:
### Measuring "average" color

# average luminance --> ?
# grayscale image and calculate average --> but what does this tell us? how is this feature valuable?

#calculating color features
colors_of_lesion = im2[mask2==1]
x_R, x_G, x_B = np.mean(colors_of_lesion, axis = 0)
print(x_R, x_G, x_B)

avg_color = (x_R + x_G + x_B)/3
print(avg_color)

In [ ]:
Perimeter = []
Area = []
Avg_color = []
Compactness = []
Asymmetry = []
Asymmetry_02 = []
Red = []
Green = []
Blue = []

for i in range(len(images)):
    im = plt.imread('../data/ISIC_2017/Images_re/'+images[i])
    mask = (plt.imread('../data/ISIC_2017/custom_masks/'+segments[i])).copy()
    
    mask[0:3,:], mask[-4:-1,:], mask[:,0:3], mask[:,-4:-1] = False,False,False,False
    
    total = mask.shape[0] * mask.shape[1] 
    area = np.sum(mask)
    
    brush = morphology.disk(3)
    
    mask_eroded = morphology.binary_erosion(mask, brush)
    
    image_perimeter = mask - mask_eroded
    perimeter = np.sum(image_perimeter)
    
    img1 = im.copy()
    img1[mask==0] = 0
    
    c = (perimeter)**2/(4*math.pi*area)
    
    array = mask.copy()
    H,W = array.shape

    left_edges = np.where(array.any(axis=1),array.argmax(axis=1),W+1)
    flip_lr = cv2.flip(array,1)
    right_edges = W - np.where(flip_lr.any(axis=1),flip_lr.argmax(axis=1),W+1)
    top_edges = np.where(array.any(axis=0),array.argmax(axis=0),H+1)
    flip_ud = cv2.flip(array,0)
    bottom_edges = H - np.where(flip_ud.any(axis=0),flip_ud.argmax(axis=0),H+1)

    leftmost = left_edges.min()
    rightmost = right_edges.max()
    topmost = top_edges.min()
    bottommost = bottom_edges.max()
    
    im2 = img1[topmost:bottommost,leftmost:rightmost,:]
    mask2 = mask[topmost:bottommost,leftmost:rightmost]

    h, w = map(int, mask2.shape)
    left = mask2[0:, 0:math.floor(w/2)]
    right = mask2[0:, math.ceil(w/2):]
    try:
        rot_im = transform.rotate(right, 180)  
    except:
        print(images[i])
        break
        
    new_im = rot_im + left
    new_im[new_im == 2] = 0

    h2, w2 = map(int, new_im.shape)
    top = new_im[0: math.floor(h2/2), 0:]
    bottom = new_im[math.ceil(h2/2):, 0:]
    rot_im2 = transform.rotate(top, 270)
    new_im2 = rot_im2 + bottom
    new_im2[new_im2 == 2] = 0
    
    asymmetry = np.sum(new_im2)/area
    

    colors_of_lesion = im2[mask2==1]
    x_R, x_G, x_B = np.mean(colors_of_lesion, axis = 0)

    avg_color = (x_R + x_G + x_B)/3
    
    Perimeter.append(perimeter)
    Area.append(area)
    Avg_color.append(avg_color)
    Compactness.append(c)
    Red.append(x_R)
    Green.append(x_G)
    Blue.append(x_B)
    Asymmetry.append(asymmetry)

In [ ]:
features = pd.read_csv("../data/ISIC_2017/ISIC-2017_Training_Part3_GroundTruth.csv")
age_and_sex = pd.read_csv("../data/ISIC_2017/ISIC-2017_Training_Data_metadata.csv")

features["Perimeter"] = Perimeter
features["Area"] = Area
features["Compactness"] = Compactness
features["Asymmetry"] = Asymmetry
features["Red"] = Red
features["Green"] = Green
features["Blue"] = Blue
features["Average Color"] = Avg_color
features["Age"] = [int(a) if a != "unknown" else None for a in age_and_sex["age_approximate"]]
features["Sex"] = [0 if s == "male" else 1 for s in age_and_sex["sex"]]
features

In [ ]:
features.to_csv("../data/ISIC_2017/features.csv", index = False)

In [ ]:
norm_features = features.copy()
for column in norm_features[['Compactness','Asymmetry','Average Color','Age']]:
    norm_features[column] = norm_features[column] /norm_features[column].abs().max()
    norm_features.columns = norm_features.columns.str.replace(column, 'Norm_'+column)
norm_features.to_csv("../data/ISIC_2017/norm_features.csv", index = False)

In [ ]:
norm_features